In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle
import csv

from gpu import set_gpu

import tensorflow as tf
from latent.ml.dl_subclass import MLP, MLPbeta, CNN, get_train, get_test
from latent.ml.dl_manual import nn_pass
import latent.session_subclass as session
import latent.utils.data_utils as prd
from latent.ml.lda import train_lda, eval_lda

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
data, params = prd.load_caps_train('home_data/TR58/DATA/MAT/20170728_103239/traindata.mat')
data = data[:,:8,:]

In [2]:
data, params = prd.load_caps_train('caps/data/traindata.mat')

In [3]:
ep = 30
n_dof = np.max(params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
mlp_beta = MLPbeta(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max = prd.prep_train_caps(data, params)

# Train neural networks
models = [mlp, mlp_beta, cnn]
models = [mlp_beta]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    train_mod = get_train()

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y in ds:
            train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train aligned LDA
y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

# mlp_enc = mlp.get_layer(name='enc')
# w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

# mlpbeta_enc = mlp_beta.get_layer(name='enc')
# w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)

# cnn_enc = cnn.get_layer(name='enc')
# temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
# temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
# w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

mlp_w = mlp.get_weights()
mlpb_w = mlp_beta.get_weights()
cnn_w = cnn.get_weights()

Epoch 1, Loss: 1.61, Accuracy: 43.04 
Epoch 30, Loss: 0.35, Accuracy: 86.87 


In [5]:
data, params = prd.load_caps_train('home_data/TR58/DATA/MAT/20170913_053927/traindata.mat')
data = data[:,:8,:].astype('float64')
y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(data, params, scaler, emg_scale)
y_test = np.hstack(y_test,np.zeros((y_test.shape[0],1)))

test_mod = get_test()
test_mod(x_test_mlp, y_test, model, test_loss, test_accuracy)
print (f'Accuracy: {test_accuracy.result()*100:.2f}')

acc = eval_lda(w, c, x_lda, y_lda)
print (f'LDA Accuracy: {acc*100:.2f}')

ValueError: in converted code:

    c:\Users\yteh\Documents\work\git\projects\latent_rep\python\latent\ml\dl_subclass.py:140 test_step  *
        t_loss = tf.keras.losses.categorical_crossentropy(y,y_out)
    C:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\tensorflow_core\python\keras\losses.py:971 categorical_crossentropy
        return K.categorical_crossentropy(y_true, y_pred, from_logits=from_logits)
    C:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\tensorflow_core\python\keras\backend.py:4504 categorical_crossentropy
        labels=target, logits=output, axis=axis)
    C:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\tensorflow_core\python\ops\nn_ops.py:3165 softmax_cross_entropy_with_logits_v2
        labels=labels, logits=logits, axis=axis, name=name)
    C:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\tensorflow_core\python\util\deprecation.py:507 new_func
        return func(*args, **kwargs)
    C:\ProgramData\Anaconda3\envs\latent_

In [4]:
# Save neural network architectures into csv
arch = ['RELU1','BN1','RELU2','BN2','RELU3','BN3','DEN','BN4','SOFTMAX']
w = {}
w_in = mlpb_w

i = 0
for l in arch:
    w_layer = []
    if 'BN' in l:
        w_layer = np.vstack((np.vstack((np.vstack((w_in[i],w_in[i+1])),w_in[i+2])),w_in[i+3]))
        i += 4
    else:
        w_layer = np.vstack((w_in[i],w_in[i+1]))
        i += 2
    with open('weights/' + l + '.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(w_layer)
    w[l] = w_layer

with open('weights/ARCH.csv', 'w', newline='') as f:
        writer = csv.writer(f,delimiter=',')
        # write multiple rows
        for i in arch:
            writer.writerow([i])

fill = len(emg_scale) - len(x_min)
temp = np.hstack((np.vstack((x_min[...,np.newaxis],np.zeros((fill,1)))), np.vstack((x_max[...,np.newaxis],np.zeros((fill,1))))))
scales = np.hstack((emg_scale,temp))

with open('weights/scales.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(scales)